### Canned Estimators

In this notebook we'll demonstrate how to use two Canned Estimators (these encapsulate the lower-level TensorFlow code we've seen so far, and use an API loosely inspired by [scikit-learn](scikit-learn.org). There are several advantages to Canned Estimators.

* If you're using Estimators, you won't have to manage Sessions, or write your own logic for TensorBoard, or for saving and loading checkpoints.


* You'll get out-of-the-box distributed training (of course, you will have to take care to read your data efficiently, and set up a cluster).

Here, we'll read data using [input functions](https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/numpy_input_fn), which are appropriate for in-memory data. 

* These provide batching and other features for you, so you don't have to write that code yourself.


* In the strucuted data notebook, we'll use the new [Dataset API](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/docs_src/programmers_guide/datasets.md) - which is a popular abstraction, and a great way to efficiently read and pre-process large datasets efficiently.

Although the Estimators we'll use here are relative simple (a LinearClassifier, and a Fully Connected Deep Neural Network), we also provide more interesting ones (including for [TensorFlow Wide and Deep](https://www.tensorflow.org/tutorials/wide_and_deep). I'm also excited that additional Estimators are on their way - stay tuned in the upcoming months.

Also note that Estimators log quite a lot of output

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

In [5]:
# We'll use Keras (included with TensorFlow) to import the data
(x_train, y_train), (x_test, y_test) = tf.contrib.keras.datasets.mnist.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = y_train.astype('int32')
y_test = y_test.astype('int32')

# Normalize the color values to 0-1
# (as imported, they're 0-255)
x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

11493376/11490434 [==============================] - 50s    
60000 train samples
10000 test samples


Here's our input function. 

* By setting ```num_epochs``` to ```None```, we'll loop over the data indefinitely so we can train for as long as we like.
* The default ```batch_size``` is ```128```, but you can provide a different parameter if you like.

You can read more about the numpy input function [here](https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/numpy_input_fn). We also provide a nice one for [Pandas](https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/pandas_input_fn), more on that later.

In [7]:
train_input = tf.estimator.inputs.numpy_input_fn(
    {'x': x_train},
    y_train, 
    num_epochs=None, # repeat forever
    shuffle=True # 
)

In [8]:
test_input = tf.estimator.inputs.numpy_input_fn(
    {'x': x_test},
    y_test,
    num_epochs=1, # loop through the dataset once
    shuffle=False # don't shuffle the test data
)

In [9]:
# define the features for our model
# the names must match the input function
feature_spec = [tf.feature_column.numeric_column('x', shape=784)]

Here, we'll create a ```LinearClassifier``` - this is identical to our Softmax (aka, multiclass logistic regression model) from the second notebok.

In [10]:
estimator = tf.estimator.LinearClassifier(feature_spec, 
                                          n_classes=10,
                                          model_dir="./graphs/canned/linear")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_model_dir': './graphs/canned/linear', '_keep_checkpoint_max': 5, '_session_config': None, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_tf_random_seed': 1}


In [11]:
# I've arbitrarily decided to train for 1000 steps
estimator.train(train_input, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./graphs/canned/linear/model.ckpt.
INFO:tensorflow:step = 1, loss = 294.731
INFO:tensorflow:global_step/sec: 558.971
INFO:tensorflow:step = 101, loss = 44.8351 (0.180 sec)
INFO:tensorflow:global_step/sec: 519.317
INFO:tensorflow:step = 201, loss = 47.181 (0.192 sec)
INFO:tensorflow:global_step/sec: 541.854
INFO:tensorflow:step = 301, loss = 36.1606 (0.185 sec)
INFO:tensorflow:global_step/sec: 562.628
INFO:tensorflow:step = 401, loss = 31.6158 (0.178 sec)
INFO:tensorflow:global_step/sec: 524.437
INFO:tensorflow:step = 501, loss = 19.4514 (0.191 sec)
INFO:tensorflow:global_step/sec: 520.441
INFO:tensorflow:step = 601, loss = 55.2783 (0.192 sec)
INFO:tensorflow:global_step/sec: 531.03
INFO:tensorflow:step = 701, loss = 38.6199 (0.188 sec)
INFO:tensorflow:global_step/sec: 460.42
INFO:tensorflow:step = 801, loss = 38.368 (0.217 sec)
INFO:tensorflow:global_step/sec: 531.628
INFO:tensorflow:step = 901, l

In [12]:
# We should see about 90% accuracy here.
evaluation = estimator.evaluate(input_fn=test_input)
print(evaluation)

INFO:tensorflow:Starting evaluation at 2018-01-18-19:48:36
INFO:tensorflow:Restoring parameters from ./graphs/canned/linear/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-01-18-19:48:36
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.9241, average_loss = 0.276041, global_step = 1000, loss = 34.9419
{'global_step': 1000, 'average_loss': 0.2760407, 'accuracy': 0.92409998, 'loss': 34.94186}


Here's how you would print individual predictions.

In [13]:
MAX_TO_PRINT = 5

# This returns a generator object
predictions = estimator.predict(input_fn=test_input)
i = 0
for p in predictions:
    true_label = y_test[i]
    predicted_label = p['class_ids'][0]
    print("Example %d. True: %d, Predicted: %d" % (i, true_label, predicted_label))
    i += 1
    if i == MAX_TO_PRINT: break

INFO:tensorflow:Restoring parameters from ./graphs/canned/linear/model.ckpt-1000
Example 0. True: 7, Predicted: 7
Example 1. True: 2, Predicted: 2
Example 2. True: 1, Predicted: 1
Example 3. True: 0, Predicted: 0
Example 4. True: 4, Predicted: 4


Here's how easy it is to switch the model to a fully connected DNN.

In [14]:
estimator = tf.estimator.DNNClassifier(
    hidden_units=[500,250], # we will arbitrarily use two layers
    feature_columns=feature_spec,
    n_classes=10,
    model_dir="./graphs/canned/deep")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_model_dir': './graphs/canned/deep', '_keep_checkpoint_max': 5, '_session_config': None, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_tf_random_seed': 1}


In [15]:
# I've arbitrarily decided to train for 2000 steps
estimator.train(train_input, steps=2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./graphs/canned/deep/model.ckpt.
INFO:tensorflow:step = 1, loss = 298.115
INFO:tensorflow:global_step/sec: 71.8219
INFO:tensorflow:step = 101, loss = 30.5674 (1.392 sec)
INFO:tensorflow:global_step/sec: 81.1383
INFO:tensorflow:step = 201, loss = 7.85995 (1.232 sec)
INFO:tensorflow:global_step/sec: 82.4658
INFO:tensorflow:step = 301, loss = 23.9902 (1.213 sec)
INFO:tensorflow:global_step/sec: 81.9004
INFO:tensorflow:step = 401, loss = 12.31 (1.221 sec)
INFO:tensorflow:global_step/sec: 81.3374
INFO:tensorflow:step = 501, loss = 22.3334 (1.230 sec)
INFO:tensorflow:global_step/sec: 81.1986
INFO:tensorflow:step = 601, loss = 13.6766 (1.232 sec)
INFO:tensorflow:global_step/sec: 80.575
INFO:tensorflow:step = 701, loss = 21.8514 (1.241 sec)
INFO:tensorflow:global_step/sec: 80.9293
INFO:tensorflow:step = 801, loss = 12.9145 (1.235 sec)
INFO:tensorflow:global_step/sec: 81.1306
INFO:tensorflow:step = 901, lo

In [16]:
# Expect accuracy around 97%
evaluation = estimator.evaluate(input_fn=test_input)
print(evaluation)

INFO:tensorflow:Starting evaluation at 2018-01-18-19:50:02
INFO:tensorflow:Restoring parameters from ./graphs/canned/deep/model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2018-01-18-19:50:03
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.9748, average_loss = 0.0823014, global_step = 2000, loss = 10.4179
{'global_step': 2000, 'average_loss': 0.082301371, 'accuracy': 0.97479999, 'loss': 10.417894}


If you like, you can compare these runs with TensorBoard.

``` $ tensorboard --logdir=graphs/canned/ ```